In [1]:
%load_ext autoreload 
%autoreload 2

import time
import os
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

import sys
sys.path.insert(0, '/home/bettyld/PJ/Documents/NLP_PJ/nlp_recommend')
from nlp_recommend.utils.utils import rerank

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bettyld/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bettyld/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bettyld/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/bettyld/PJ/Documents/NLP_PJ/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [30]:
from nlp_recommend import LoadData, clean_text

DATASET = 'psychology'

pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 500

# corpus = LoadData(dataset=DATASET,random=False, remove_numbered_rows=True, cache=False)
corpus = LoadData(dataset=DATASET, cache=True) 
philo_df = corpus.corpus_df

philo_df = philo_df[['sentence', 'author', 'title', 'tok_lem_sentence']]
philo_df['clean_sentence'] = philo_df['sentence'].apply(lambda x: clean_text(x, only_symbols=True))

print('number of sentences:', len(philo_df))
philo_df.head()

number of sentences: 11483


/home/bettyld/PJ/Documents/NLP_PJ/env/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,sentence,author,title,tok_lem_sentence,clean_sentence
level_0,,,,,
523,For every little boy the father gets in the way,Jackson,Outwitting Our Nerves: A Primer of Psychotherapy,"[every, little, boy, father, get, way]",for every little boy the father gets in the way
524,For every little girl the mother gets in the way,Jackson,Outwitting Our Nerves: A Primer of Psychotherapy,"[every, little, girl, mother, get, way]",for every little girl the mother gets in the way
525,"At one time or other there is likely to be a' ""period when this is resented with all the violence of a child's"" 'emotions",Jackson,Outwitting Our Nerves: A Primer of Psychotherapy,"[one, time, likely, period, resented, violence, child, emotion]",at one time or other there is likely to be a period when this is resented with all the violence of a child emotions
526,It is likely to be very soon repressed and succeeded by a real affection which lasts through life,Jackson,Outwitting Our Nerves: A Primer of Psychotherapy,"[likely, soon, repressed, succeeded, real, affection, last, life]",it is likely to be very soon repressed and succeeded by a real affection which lasts through life
528,Jealousy is very often united with the natural rebellion of a child against authority,Jackson,Outwitting Our Nerves: A Primer of Psychotherapy,"[jealousy, often, united, natural, rebellion, child, authority]",jealousy is very often united with the natural rebellion of a child against authority


In [31]:
print(len(philo_df.author.unique()), len(philo_df.title.unique()))

20 25


In [32]:
philo_df[['title', 'author']].drop_duplicates().sort_values(by=['author']).to_csv(f'{DATASET}_author_title.csv')